In [125]:
# Notebook cell 1
import sys, pathlib
from tools.prompt_cache import cached_call

# add the project root so Python can see `personaltrainers/`
ROOT = pathlib.Path.cwd()          # change if you launch the notebook elsewhere
sys.path.append(str(ROOT))

# reload local modules each time you run the cell (handy while editing)
from crew import Personaltrainers
from tools.db_call import db_call


# 1: SQL FILTER GENERATOR
def sql_filter(client_request: str) -> str:
    inputs = {"user_request": client_request}
    result = Personaltrainers().crew().kickoff(inputs=inputs).raw
    result = result.replace("`", "")
    result = result.replace("\n", "")

    sql_filter = " ".join(result.split())

    return sql_filter

In [171]:
import json
from pathlib import Path

# ---------------- Helper functions ---------------- #
def prompt_choice(question: str, options: list[str]) -> str:
    """Display a question with enumerated options and return the selected value."""
    print(question)
    for idx, opt in enumerate(options, start=1):
        print(f"  {idx}. {opt}")
    while True:
        choice = input("Select the corresponding number: ").strip()
        if choice.isdigit() and 1 <= int(choice) <= len(options):
            return options[int(choice) - 1]
        print("Invalid input, please try again.")

def prompt_yes_no(question: str) -> bool:
    """Return True if the user answers 'y/Y', False if 'n/N'."""
    while True:
        ans = input(f"{question} [y/n]: ").strip().lower()
        if ans in ("y", "n"):
            return ans == "y"
        print("Please answer with 'y' for yes or 'n' for no.")

def prompt_int_range(question: str, minimum: int, maximum: int) -> int:
    """Ask for an integer within [minimum, maximum] (inclusive)."""
    while True:
        ans = input(f"{question} ({minimum}-{maximum}): ").strip()
        if ans.isdigit() and minimum <= int(ans) <= maximum:
            return int(ans)
        print("Number out of range, please try again.")

def prompt_multi_select(options: list[str]) -> list[str]:
    """
    Let the user select multiple items by number (comma-separated).
    Return the chosen items as a list.
    """
    for idx, opt in enumerate(options, start=1):
        print(f"  {idx}. {opt}")
    while True:
        raw = input("Enter the corresponding numbers (comma-separated): ").replace(" ", "")
        indices = [s for s in raw.split(",") if s.isdigit()]
        if all(1 <= int(i) <= len(options) for i in indices):
            return [options[int(i) - 1] for i in indices] if indices else []
        print("Invalid selection, please try again.")

# ---------------- Questionnaire ---------------- #
def run_questionnaire() -> dict:
    """Run all steps and return a dict representing the payload."""
    # 1. Main workout focus
    train_question = "What do you want to train today?"
    train_options  = ["Upper body", "Lower body", "Full body"]
    train_target = prompt_choice(train_question, train_options)

    # 2. Specific muscle focus (conditional)
    focus_muscle = prompt_yes_no("Do you want to put specific focus on certain muscles?")
    muscles = []
    if focus_muscle:
        # Map allowed muscles by primary focus
        muscle_map = {
            "Upper body": ["Chest", "Lats", "Shoulders", "Biceps", "Triceps"],
            "Lower body": ["Quads", "Hamstrings", "Glutes", "Calves"],
            "Full body" : ["Chest", "Back", "Shoulders", "Biceps", "Triceps",
                           "Quads", "Hamstrings", "Glutes", "Calves"],
        }
        available = muscle_map[train_target]
        print("Select the muscle groups (comma-separated):")
        for idx, mus in enumerate(available, start=1):
            print(f"  {idx}. {mus}")
        while True:
            raw = input("Enter the corresponding numbers: ").replace(" ", "")
            indices = [s for s in raw.split(",") if s.isdigit()]
            if all(1 <= int(i) <= len(available) for i in indices):
                muscles = [available[int(i)-1] for i in indices]
                break
            print("Invalid selection, please try again.")

    # 3. Time available
    minutes = prompt_int_range("How much time do you have available (in minutes)?", 45, 180)

    # 4. Training location
    location = prompt_choice("Where are you training?", ["Gym", "Park", "Home"])

    # 5. Equipment available (conditional)
    equipment_list = ["Gymnastics rings", "Parallel bars", "Parallettes",
                      "Pull-up bar", "Barbell", "Dumbbells", "Kettlebell"]
    equipment_available = False
    equipment = []
    if location in {"Park", "Home"}:
        equipment_available = prompt_yes_no(
            "Do you have any specific equipment available?"
        )
        if equipment_available:
            print("Select the equipment you have (leave blank for none):")
            equipment = prompt_multi_select(equipment_list)

    # Assemble payload
    payload = {
        "train_target": train_target,
        "focus_muscle": focus_muscle,
        "muscles": muscles,                   # empty list if none
        "duration_minutes": minutes,
        "location": location,
        "equipment_available": equipment_available,
        "equipment": equipment                # empty list if none
    }
    return payload

# ---------------- Main entry point ---------------- #
answers = run_questionnaire()
file_path = Path("workout_payload.json")
with file_path.open("w", encoding="utf-8") as fp:
    json.dump(answers, fp, ensure_ascii=False, indent=2)
print("\n--- Answers saved! ---")
print(json.dumps(answers, ensure_ascii=False, indent=2))
print(f"\nJSON file created: {file_path.resolve()}")

What do you want to train today?
  1. Upper body
  2. Lower body
  3. Full body
Select the muscle groups (comma-separated):
  1. Chest
  2. Back
  3. Shoulders
  4. Biceps
  5. Triceps
  6. Quads
  7. Hamstrings
  8. Glutes
  9. Calves
Where are you training?
  1. Gym
  2. Park
  3. Home
Invalid input, please try again.

--- Answers saved! ---
{
  "train_target": "Full body",
  "focus_muscle": true,
  "muscles": [
    "Chest",
    "Back",
    "Quads"
  ],
  "duration_minutes": 120,
  "location": "Park",
  "equipment_available": false,
  "equipment": []
}

JSON file created: C:\Users\Emanuele\Desktop\LazyTrainer\personaltrainers\src\personaltrainers\workout_payload.json


In [172]:
with file_path.open("r", encoding="utf-8") as fp:
    answers = json.load(fp)

answers

{'train_target': 'Full body',
 'focus_muscle': True,
 'muscles': ['Chest', 'Back', 'Quads'],
 'duration_minutes': 120,
 'location': 'Park',
 'equipment_available': False,
 'equipment': []}

In [177]:
if answers.get('train_target') == 'Full body':
    answers['train_target'] = 'Upper body, Lower body'

if answers.get('focus_muscle'):
    prompt = f"""
    Select the exercises using these informations: 
    - body_region: {answers.get('train_target')}, 
    - muscles: {answers.get('muscles')},
    - load_class: {answers.get('location')}"""
else:
    prompt = f"""
    Select the exercises using these informations: 
    - body_region: {answers.get('train_target')},
    - load_class: {answers.get('location')}"""

if answers.get('equipment_available'):
    prompt += f"""
    - equipment: Ground, {answers.get('equipment')}"""
prompt = prompt.replace("'", "")
prompt = prompt.replace("[", "")
prompt = prompt.replace("]", "")
print(prompt)


    Select the exercises using these informations: 
    - body_region: Upper body, Lower body, 
    - muscles: Chest, Back, Quads,
    - load_class: Park


In [178]:
filter_generated = cached_call(prompt, sql_filter)
exercises_df = db_call(filter_generated)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 7bc61d14-e616-4a71-8379-3ae0521b6e36                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 6ac90fd2-4255-4699-981f-5bad23cb3fe4
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 6ac90fd2-4255-4699-981f-5bad23cb3fe4
       Status: Executing Task...
    └── 🤖 Agent: Personal Trainer AI
        
            Status: In Progress

# Agent: Personal Trainer AI
## Task: The user says: **
    Select the exercises using these informations: 
    - body_region: Upper body, Lower body, 
    - muscles: Chest, Back, Quads,
    - load_class: Park**.
Your task is to generate a WHERE filter for an SQL query based on the user request \ if the request does not speak about a training program your output mut be empty string.
Your output must be an SQL query like the one delimited by triple backtics. \ If a column has no values drop it
WHERE
    body_region IN () AND 
    (
    load_class LIKE '%load_class%' OR
    load_class LIKE '%load_class%'
    OR equipments in ()
    )
    AND
    (
    muscles LIKE '%muscle_name%' OR
    muscles LIKE '%muscle_name%' OR 
    )
    ... 


Possible values:
body_region: 'upper body', 'lower body'  muscles: 'chest', 'shoulders', 'tricpes', 'lats', 'biceps', 'quads', 'harmstrings', 'glutes', 'calves' load_class: 'bodyweight', 'free-weight', 'machine' equipments: 'gymnastics rings, parallel bars

🚀 Crew: crew
└── 📋 Task: 6ac90fd2-4255-4699-981f-5bad23cb3fe4
       Status: Executing Task...
    └── 🤖 Agent: Personal Trainer AI
        
            Status: ✅ Completed

🚀 Crew: crew
└── 📋 Task: 6ac90fd2-4255-4699-981f-5bad23cb3fe4
       Assigned to: Personal Trainer AI
    
       Status: ✅ Completed
    └── 🤖 Agent: Personal Trainer AI
        
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 6ac90fd2-4255-4699-981f-5bad23cb3fe4                                                                     │
│  Agent: Personal Trainer AI                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 7bc61d14-e616-4a71-8379-3ae0521b6e36                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [179]:
exercises_df

,id,name,movement_pattern,movement_type,body_region,load_class,muscles,equipments
0,3,parallel bar dips,horizontal push,compound,upper body,bodyweight,"chest, shoulders, triceps","gymnastics rings, parallel bars"
1,4,push ups,horizontal push,compound,upper body,bodyweight,"chest, shoulders, triceps","ground, gymnastics rings, parallettes"
2,16,squat b,legs,compound,lower body,bodyweight,"glutes, quads",ground
3,18,squat jump b,legs,compound,lower body,bodyweight,"glutes, quads",ground
4,20,lunges b,legs,compound,lower body,bodyweight,"glutes, quads",ground
5,22,bulgarian squat b,legs,compound,lower body,bodyweight,"glutes, quads",ground


In [169]:
exercises_df.to_dict(orient="records")

[{'id': 3,
  'name': 'parallel bar dips',
  'movement_pattern': 'horizontal push',
  'movement_type': 'compound',
  'body_region': 'upper body',
  'load_class': 'bodyweight',
  'muscles': 'chest, shoulders, triceps',
  'equipments': 'gymnastics rings, parallel bars'},
 {'id': 4,
  'name': 'push ups',
  'movement_pattern': 'horizontal push',
  'movement_type': 'compound',
  'body_region': 'upper body',
  'load_class': 'bodyweight',
  'muscles': 'chest, shoulders, triceps',
  'equipments': 'ground, gymnastics rings, parallettes'},
 {'id': 8,
  'name': 'v-push ups',
  'movement_pattern': 'vertical push',
  'movement_type': 'compound',
  'body_region': 'upper body',
  'load_class': 'bodyweight',
  'muscles': 'shoulders, triceps',
  'equipments': 'ground'}]